# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
from vaastav.getters import *
from datetime import datetime

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchvision

import torch
import torch.nn as nn

# Constants

In [3]:
NEXT_GW = 25

# fixtures in sample
FIS = 8

# prediction range
FROM_GW = 25
TO_GW = 25

batch_size = 16

# Device

In [4]:
# device = torch.device("cuda")
# device = torch.device('mps')
device = torch.device('cpu')

# Reading data

In [5]:
upcoming_fixtures = pd.read_csv("data/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (3,4,5,6,9,10,37,40,41,45,48,50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# selecting needed rows
upcoming_fixtures = upcoming_fixtures[(upcoming_fixtures["Finished"] == True) | ((upcoming_fixtures["Finished"] == False) & (upcoming_fixtures["GW"].between(FROM_GW, TO_GW)))]

In [7]:
# upcoming_fixtures[upcoming_fixtures["Name"] == "Erling-Haaland"]

In [8]:
# upcoming_fixtures.loc[9315]

# Dataset

In [9]:
features = ["Min", "Gls", "Ast", "Sh", "SoT", "xG", "npxG", "xAG", "SCA", "GCA", "CS", "CrdY", "CrdR", "Opp rating", "Team Score", "Opp Score", "Was Home"]

In [10]:
def init_index_table(df):
    names = df["Name"].unique()
    result_dict = {}
    
    current_index = 0
    player_index = 0
    
    for name in names:
        sample_index = 0
        samples_for_name = df[(df["Name"] == name) & (df["Finished"] == False)].shape[0]
        for i in range(samples_for_name):
            result_dict[current_index] = [player_index, sample_index]
            current_index += 1
            sample_index += 1
        player_index += 1
    
    return result_dict

In [11]:
class PandasDataset(Dataset):
    def __init__(self, dataframe, starting_gw, ending_gw):
        self.dataframe = dataframe
        
        self.starting_gw = starting_gw
        self.ending_gw = ending_gw
        self.names = self.dataframe["Name"].unique()
        self.length = self.calculate_length()

        self.index_table = init_index_table(self.dataframe)
        self.transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

    def calculate_length(self):
        length = 0
        for name in self.names:
            # print(name, (self.dataframe[self.dataframe["Name"] == name].shape[0] - FIS))
            length += self.dataframe[(self.dataframe["Name"] == name) & self.dataframe["GW"].between(FROM_GW, TO_GW)].shape[0]
        return length
    
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # print(idx)
        player_id, sample_id = self.index_table[idx]
        name = self.names[player_id]
        # print(name)
        # print("GETITEM DF", name, player_id, sample_id)

        rows_features = self.dataframe[(self.dataframe["Name"] == name) & (self.dataframe["Finished"] == True)]
        # print(self.dataframe[self.dataframe["Name"] == name])
        # print(features[sample_id: (sample_id + 4)])
        features = rows_features.tail(FIS)[features + to_predict].values[:, :-1]
        # print(features)
        # print(features)
        # print(type(features))
        # print(self.dataframe[self.dataframe["Name"] == name].values[:, -1])
        # print(type(self.dataframe[self.dataframe["Name"] == name].values[:, -1]))
        # target = self.dataframe[self.dataframe["Name"] == name].values[:, -1]
        # target = target.astype(np.float32)
        # target = torch.from_numpy(target)
        # print("GETITEM", idx, player_id, sample_id, name)
        # print(name, idx, target, sample_id)
        # print(type(target))
        return features

In [12]:
dataset = PandasDataset(upcoming_fixtures, FROM_GW, TO_GW)

In [13]:
dataset.__len__()

438

In [14]:
dataloader_general_test = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Models

In [15]:
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.transpose(x, 0, 1)
        # x = x.view(-1, batch_size)
        # x = torch.transpose(x, 0, 1)
        # print("FOR", x.shape)
        # print(x.shape)
        # print(x.shape)
        out, _ = self.lstm(x)
        # print(out.shape)
        out = self.fc(out[-1])
        return out

In [16]:
net = LSTMNetwork(1, 20, 2, 1, dropout=0.4).to(device)
net.load_state_dict(torch.load('models/best/best_lstm_general.pt'))

FileNotFoundError: [Errno 2] No such file or directory: 'models/best/best_lstm_general.pt'